__Задание 1: взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)__

__Решение:__ выбрал Shill Bidding Dataset, скачал его залил в свой коллаб

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [3]:
with open('/content/drive/MyDrive/ГБ/выборки для исследований/Shill Bidding Dataset.csv', "rb") as f:
    data = pd.read_csv(f)

Attribute Information:

Record ID: Unique identifier of a record in the dataset. <br>
Auction ID: Unique identifier of an auction. <br>
Bidder ID: Unique identifier of a bidder. <br>
Bidder Tendency: A shill bidder participates exclusively in auctions of few sellers rather than a diversified lot. This is a collusive act involving the fraudulent seller and an accomplice. <br>
Bidding Ratio: A shill bidder participates more frequently to raise the auction price and attract higher bids from legitimate participants. <br>
Successive Outbidding: A shill bidder successively outbids himself even though he is the current winner to increase the price gradually with small consecutive increments. <br>
Last Bidding: A shill bidder becomes inactive at the last stage of the auction (more than 90\% of the auction duration) to avoid winning the auction. <br>
Auction Bids: Auctions with SB activities tend to have a much higher number of bids than the average of bids in concurrent auctions. <br>
Auction Starting Price: a shill bidder usually offers a small starting price to attract legitimate bidders into the auction. <br>
Early Bidding: A shill bidder tends to bid pretty early in the auction (less than 25\% of the auction duration) to get the attention of auction users. <br>
Winning Ratio: A shill bidder competes in many auctions but hardly wins any auctions. <br>
Auction Duration: How long an auction lasted.

__Задание 2: сделать feature engineering__

__Решение__

In [4]:
# смотрим данные
data

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.000000,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.000000,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.000000,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.000000,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.000000,0.000000,0.001242,0.500000,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6316,15129,760,l***t,0.333333,0.160000,1.0,0.738557,0.280000,0.993593,0.686358,0.888889,3,1
6317,15137,2481,s***s,0.030612,0.130435,0.0,0.005754,0.217391,0.993593,0.000010,0.878788,7,0
6318,15138,2481,h***t,0.055556,0.043478,0.0,0.015663,0.217391,0.993593,0.015663,0.000000,7,0
6319,15139,2481,d***d,0.076923,0.086957,0.0,0.068694,0.217391,0.993593,0.000415,0.000000,7,0


In [5]:
# видим что первые три колонки не имеют ценности для нашей будущей модели МО
data = data.drop(columns=['Record_ID',	'Auction_ID',	'Bidder_ID'])

In [6]:
# смотрим пропуски и типы данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6321 entries, 0 to 6320
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Bidder_Tendency         6321 non-null   float64
 1   Bidding_Ratio           6321 non-null   float64
 2   Successive_Outbidding   6321 non-null   float64
 3   Last_Bidding            6321 non-null   float64
 4   Auction_Bids            6321 non-null   float64
 5   Starting_Price_Average  6321 non-null   float64
 6   Early_Bidding           6321 non-null   float64
 7   Winning_Ratio           6321 non-null   float64
 8   Auction_Duration        6321 non-null   int64  
 9   Class                   6321 non-null   int64  
dtypes: float64(8), int64(2)
memory usage: 494.0 KB


Нам повезло и пропусков в датасете нет (все строки заполнены), а также и с типом данных - ничего преобразовывать не нужно - все данные числовые

In [7]:
# посмотрим наличе строк-дубликатов
data[data.duplicated()]

,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
3311,0.300000,0.300000,0.0,0.010230,0.0,0.000000,0.007149,0.5,7,0
5236,0.055556,0.062500,0.0,0.000027,0.0,0.993593,0.000027,0.0,3,0
5771,0.000000,0.111111,0.0,0.000025,0.0,0.000000,0.000025,0.0,5,0


Дубликатов нет.

__Задание 3: обучить любой классификатор (какой вам нравится)__

__Решение__ Буду использовать лог. регрессию, построю модель без подбора гиперпараметров и посмотрю качество ее работы по метрикам. Из этого станет понятно нужно ли тратить время на преобразование признаков (полиномы и тп) и подбор гиперпараметров у модели МО

In [8]:
y = data.Class
data.drop(columns='Class', inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data, y, random_state=1)

In [9]:
lg = LogisticRegression(random_state=1).fit(X_train, y_train)

preds_proba = lg.predict_proba(X_test)[:, 1]



results = pd.DataFrame(columns=['thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])


# precision, recall, thresholds = precision_recall_curve(y_test, preds_proba)
# fscore = (2 * precision * recall) / (precision + recall)
# auc = roc_auc_score(y_test, preds_proba)
# ix = np.argmax(fscore)

# results = results.append({
#   'thresh': thresholds[ix],
#   'F-Score': fscore[ix],
#   'Precision': precision[ix],
#   'Recall': recall[ix],
#   'ROC AUC': auc
#   }, ignore_index=True)

# results


def compute_result(y_true, preds_proba, result):
    precision, recall, thresholds = precision_recall_curve(y_true, preds_proba)
    fscore = (2 * precision * recall) / (precision + recall)
    auc = roc_auc_score(y_true, preds_proba)
    ix = np.nanargmax(fscore)          # np.argmax(fscore) не подходит, т.к. в массиве значений фмеры иногда всречаются nan

    result = result.append({
      'thresh': thresholds[ix],
      'F-Score': fscore[ix],
      'Precision': precision[ix],
      'Recall': recall[ix],
      'ROC AUC': auc
      }, ignore_index=True)
    
    return result

results = compute_result(y_test, preds_proba, results)
results

,thresh,F-Score,Precision,Recall,ROC AUC
0,0.282067,0.910615,0.840206,0.993902,0.996179


Модель лог.регрессии с гиперпараметрами по умолчанию выдает прекрасные показатели.

В целом с данными порядок, предсказания делаем достоверно. Можем перейти к тестированию работы метода look alike вместо лог. регрессии.

<br>

__Задание 4:  далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть__

In [11]:
# для начала посмотрим баланс классов. Видим дисбаланс:
y.value_counts()

0    5646
1     675
Name: Class, dtype: int64

In [12]:
# сделаем копию 
all_data = data.copy()

Сначала попробую взять 30% данных, соответствующих классу 1, потом можно будет поэкспериментировать с другими пропорциями

In [13]:
n = int(675 * 0.3)
n

202

In [14]:
all_data['y'] = y
all_data.head(3)

,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,y
0,0.200000,0.4,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
1,0.024390,0.2,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
2,0.142857,0.2,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0


In [15]:
all_possitive_indx = list(all_data.loc[all_data.y == 1].index)

# пермешиваем
np.random.shuffle(all_possitive_indx)

# берем n первых позитивных индексов в перемешанном списке
pos_sample_idx = all_possitive_indx[:n]

len(pos_sample_idx)


202

Теперь приступим к изменению разметки в датасете. По условию задачи из всех позитивных классов нужно было выбрать часть, а все остальное отметить как unlabled

In [16]:
all_data.loc[pos_sample_idx,'y'] = 'P'
all_data.loc[(all_data["y"] == 0) | (all_data["y"] == 1), "y"] = 'U'

all_data.rename(columns={'y' : 'label'}, inplace=True)

In [17]:
all_data.label.value_counts()


U    6119
P     202
Name: label, dtype: int64

In [18]:
all_data

,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,label
0,0.200000,0.400000,0.0,0.000028,0.000000,0.993593,0.000028,0.666667,5,U
1,0.024390,0.200000,0.0,0.013123,0.000000,0.993593,0.013123,0.944444,5,U
2,0.142857,0.200000,0.0,0.003042,0.000000,0.993593,0.003042,1.000000,5,U
3,0.100000,0.200000,0.0,0.097477,0.000000,0.993593,0.097477,1.000000,5,U
4,0.051282,0.222222,0.0,0.001318,0.000000,0.000000,0.001242,0.500000,7,U
...,...,...,...,...,...,...,...,...,...,...
6316,0.333333,0.160000,1.0,0.738557,0.280000,0.993593,0.686358,0.888889,3,U
6317,0.030612,0.130435,0.0,0.005754,0.217391,0.993593,0.000010,0.878788,7,U
6318,0.055556,0.043478,0.0,0.015663,0.217391,0.993593,0.015663,0.000000,7,U
6319,0.076923,0.086957,0.0,0.068694,0.217391,0.993593,0.000415,0.000000,7,U


__Задание 5: применить random negative sampling для построения классификатора в новых условиях__

__Решение__

In [19]:
all_data.label = np.where((all_data.label == 'P'), 1, -1) 
all_data.label.value_counts()

-1    6119
 1     202
Name: label, dtype: int64

In [20]:
# Часть 1 выделяем выбранные 202 позитивных наблюдения из датасета
y_train_P = all_data.loc[all_data["label"] == 1].label


# Часть 2 выделяем  202 неразмеченных наблюдения из датасета (будет обучать модель на сбалансированных данных)
# сначала отделим индексы от позитивных
unlabled_indx = [i for i in all_data.index if i not in pos_sample_idx]
# перемемаем их чтобы выбрать случайным образом
np.random.shuffle(unlabled_indx)

# выделим идексы для трейна U и все остальные для теста
indx_train_U = unlabled_indx[:202]
indx_test = unlabled_indx[202:]
y_train_U = all_data.loc[indx_train_U,'label']

# соединяем позитив и неразмеченные метки для трейна - 404 шт итого
y_train = pd.concat([y_train_P, y_train_U]).sample(frac=1)

# выделяем данные для y_test
y_test = all_data.loc[indx_test ,'label']

X_train = all_data.loc[y_train.index, :].drop(columns='label')
X_test = all_data.loc[indx_test, :].drop(columns='label')

In [21]:
lg2 = LogisticRegression(random_state=1).fit(X_train, y_train)

preds_proba = lg2.predict_proba(X_test)[:, 1]



__Задание 6: сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)__

__Решение__

In [22]:
# наш y_test не подходит, т.к. там ведь неразмеченные данные - все один класс нудевой, потому возьмем по индексам из оригинального 'y'
y_test_for_metrics = y.loc[indx_test]



results = compute_result(y_test_for_metrics, preds_proba, results)
results

,thresh,F-Score,Precision,Recall,ROC AUC
0,0.282067,0.910615,0.840206,0.993902,0.996179
1,0.591312,0.884729,0.810469,0.973970,0.995912


__Задание 7: поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)__

__Решение:__

In [26]:
percent_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

# обнуляем результаты
results = pd.DataFrame(columns=['percent of P','thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])



In [28]:
for i in percent_list:
    new_data = data.copy()
    new_data['y'] = y

    n = int(675 * i)

    all_possitive_indx = list(new_data.loc[new_data.y == 1].index)
    # пермешиваем
    np.random.shuffle(all_possitive_indx)

    # берем n первых позитивных индексов в перемешанном списке
    pos_sample_idx = all_possitive_indx[:n]

    new_data.loc[pos_sample_idx,'y'] = 'P'
    new_data.loc[(new_data["y"] == 0) | (new_data["y"] == 1), "y"] = 'U'

    new_data.rename(columns={'y' : 'label'}, inplace=True)
    new_data.label = np.where((new_data.label == 'P'), 1, -1) 

    # Часть 1 выделяем выбранные 202 позитивных наблюдения из датасета
    y_train_P = new_data.loc[new_data["label"] == 1].label


    # Часть 2 выделяем  202 неразмеченных наблюдения из датасета (будет обучать модель на сбалансированных данных)
    # сначала отделим индексы от позитивных
    unlabled_indx = [i for i in new_data.index if i not in pos_sample_idx]
    # перемемаем их чтобы выбрать случайным образом
    np.random.shuffle(unlabled_indx)

    # выделим идексы для трейна U и все остальные для теста
    indx_train_U = unlabled_indx[:202]
    indx_test = unlabled_indx[202:]
    y_train_U = all_data.loc[indx_train_U,'label']

    # соединяем позитив и неразмеченные метки для трейна - 404 шт итого
    y_train = pd.concat([y_train_P, y_train_U]).sample(frac=1)

    # выделяем данные для y_test
    y_test = new_data.loc[indx_test ,'label']

    X_train = new_data.loc[y_train.index, :].drop(columns='label')
    X_test = new_data.loc[indx_test, :].drop(columns='label')

    lg2 = LogisticRegression(random_state=1).fit(X_train, y_train)

    preds_proba = lg2.predict_proba(X_test)[:, 1]

    y_test_for_metrics = y.loc[indx_test]


    precision, recall, thresholds = precision_recall_curve(y_test_for_metrics, preds_proba)
    fscore = (2 * precision * recall) / (precision + recall)
    auc = roc_auc_score(y_test_for_metrics, preds_proba)
    ix = np.nanargmax(fscore)          # np.argmax(fscore) не подходит, т.к. в массиве значений фмеры иногда всречаются nan

    results = results.append({
        'percent of P': i,
        'thresh': thresholds[ix],
        'F-Score': fscore[ix],
        'Precision': precision[ix],
        'Recall': recall[ix],
        'ROC AUC': auc
        }, ignore_index=True)


results





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in true_divide


,percent of P,thresh,F-Score,Precision,Recall,ROC AUC
0,0.1,0.223861,0.904239,0.834783,0.986301,0.995539
1,0.2,0.596875,0.899341,0.885185,0.913958,0.995671
2,0.3,0.444027,0.885437,0.797203,0.995633,0.995931
3,0.4,0.787917,0.865409,0.853598,0.877551,0.995287
4,0.5,0.778462,0.863702,0.826923,0.903904,0.996265
5,0.6,0.816854,0.829358,0.790210,0.872587,0.995757
6,0.7,0.901739,0.808717,0.769585,0.852041,0.995718


На будущее сделать сравнение с 2-step approach 